# 2. Eigengesichter

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext version_information

### a) Laden Sie sich den oben angegebenen Datensatz herunter. Erstellen Sie ein Python- Skript, dass die Verzeichnisse des Datensatzes durchsucht und die Personen ermittelt, für die mindestens 70 Bilder existieren. Die dafür geeigneten Funktionen finden sich im Standardmodul os bzw. os.path.

In [3]:
import os
import tarfile
from urllib.request import urlretrieve

lfw_filename = 'lfw-funneled.tgz'
lfw_directory = '.lfw-dataset'

if not os.path.isfile(lfw_filename):
    print("Downloading")
    urlretrieve('http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz',filename = lfw_filename)


if not os.path.isdir(lfw_directory):
    # Dateien in das Zielverzeichnis extrahieren
    with tarfile.open(lfw_filename, 'r:gz') as tar:
        tar.extractall(path=lfw_directory)

min_images_required = 70
selected_persons = []
extracted_path = ".lfw-dataset/lfw_funneled"

# Verzeichnis durchsuchen
for person_folder in os.listdir(extracted_path):
    person_path = os.path.join(extracted_path, person_folder)

    if os.path.isdir(person_path):
        # Anzahl der Bilder fuer die aktuelle Person zaehlen
        num_images = len([f for f in os.listdir(person_path) if f.endswith('.jpg')])

        # Ueberpruefen, ob Mindestanzahl erreicht
        if num_images >= min_images_required:
            selected_persons.append({
                'person_name': person_folder,
                'num_images': num_images
            })

print(f"Personen mit mindestens {min_images_required} Bildern:")
for person_info in selected_persons:
    print(f"{person_info['person_name']}: {person_info['num_images']} Bilder")

Personen mit mindestens 70 Bildern:
Ariel_Sharon: 77 Bilder
Colin_Powell: 236 Bilder
Donald_Rumsfeld: 121 Bilder
George_W_Bush: 530 Bilder
Gerhard_Schroeder: 109 Bilder
Hugo_Chavez: 71 Bilder
Tony_Blair: 144 Bilder


### b) Erstellen Sie ein Python-Skript, das alle Bilder bis auf eines pro Person (diese werden später zum Testen des Klassifikators gebraucht) dieser am häufigsten abgebildeten Personen lädt, diese in Vektoren stackt und dann in einer gemeinsamen Designmatrix ablegt. Zum Laden der Bilder in Numpy-Arrays verwenden Sie am einfachsten das Modul scikit-image. Schneiden Sie zunächst einen einheitlichen zentralen Ausschnitt aus, der nur Augen und Mund enthält. Skalieren Sie die Bilder auf die Größe 32 × 32. Achten Sie darauf, vorher die Farbbilder in Grauwerte umzuwandeln (z.B. mit der Option as_gray = True) Legen Sie zusätzlich einen Vektor an, in dem der Name der Person (d.h. der Ordnername) für jede Zeile steht. Führen Sie die gleiche Art der Verarbeitung mit dem übrig gebliebenen Testbild pro Person durch und speichern Sie diese getrennt ab.

In [4]:
from skimage import io, transform
dataset_path    = extracted_path
train_data_path = ".lfw-dataset-train"
test_data_path  = ".lfw-dataset-test"
os.makedirs(train_data_path, exist_ok=True)
os.makedirs(test_data_path, exist_ok=True)

# Trainingsdaten verarbeiten
for person in selected_persons:
    person_folder = person['person_name']
    person_path = os.path.join(dataset_path, person_folder)
    train_person_data = []

    # Bilder laden
    for filename in os.listdir(person_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(person_path, filename)
            # Laden und in Graustufen konvertieren
            img = io.imread(image_path, as_gray=True)
            # Zuschneiden und skalieren auf 32x32
            img = transform.resize(img[50:150, 50:150], (32, 32))
            # In einen Vektor packen
            img_vector = img.flatten()
            train_person_data.append(img_vector)

    # Daten in einer gemeinsamen Designmatrix speichern
    train_person_data = np.array(train_person_data)
    np.save(os.path.join(train_data_path, f"{person_folder}_train.npy"), train_person_data)

# Testdaten
for person in selected_persons:
    person_folder = person['person_name']
    person_path = os.path.join(dataset_path, person_folder)
    test_person_data = []

    test_image_path = os.path.join(person_path, f"{person_folder}_0001.jpg")
    # Laden und in Graustufen konvertieren
    img = io.imread(test_image_path, as_gray=True)
    # Zuschneiden und skalieren auf 32x32
    img = transform.resize(img[50:150, 50:150], (32, 32))
    # In einen Vektor packen
    img_vector = img.flatten()
    test_person_data.append(img_vector)

    # Daten getrennt abspeichern
    test_person_data = np.array(test_person_data)
    np.save(os.path.join(test_data_path, f"{person_folder}_test.npy"), test_person_data)

### c) Wenden Sie nun Ihre Hauptkomponentenanalyse aus Arbeitsblatt 1 auf Ihre Designmatrix (Achtung: kopieren Sie alle Trainingsbilder für alle Personen als Zeilen in eine gemeinsame Designmatrix!) an. Stellen Sie die ersten 150 Eigenwerte in einem Diagramm und die ersten 12 Eigengesichter durch Umformung der gestackten Darstellung in das ursprüngliche Bildformat dar. Interpretieren Sie das Ergebnis.

In [8]:
train_data = []

# Durchsuchen Sie das Verzeichnis und laden Sie die Matrizen
for filename in os.listdir(train_data_path):
    train_person_data = np.load(os.path.join(train_data_path, filename))
    train_data.append(train_person_data)

X = pd.DataFrame(np.concatenate(train_data, axis=0))

from pca import pca
num_components = len(X.columns)
X_pca, Sigma, V = pca(X, num_components)

# Umrechnung von Singulärwerten in Eigenwerte
eigenvalues = (Sigma**2) / (len(X) - 1)

# Gesamtvarianz
total_variance = sum(eigenvalues)

# Erklärte Varianz für jede Komponente
explained_variances = [(i / total_variance) * 100 for i in eigenvalues]

# Kumulative erklärte Varianz
cumulative_variances = np.cumsum(explained_variances)

# Tabellarische Darstellung
results = pd.DataFrame({
    'Eigenwert': eigenvalues,
    'Erklärte Varianz (%)': explained_variances,
    'Kumulative erklärte Varianz (%)': cumulative_variances
})

print("Ergebnisse der PCA-Analyse:")
print(results.head(150))


Ergebnisse der PCA-Analyse:
      Eigenwert  Erklärte Varianz (%)  Kumulative erklärte Varianz (%)
0    257.423576             25.119503                        25.119503
1    185.498152             18.100989                        43.220493
2     84.995729              8.293920                        51.514412
3     75.442555              7.361717                        58.876129
4     36.440521              3.555882                        62.432011
..          ...                   ...                              ...
145    0.286574              0.027964                        97.204736
146    0.283656              0.027679                        97.232416
147    0.277384              0.027067                        97.259483
148    0.274706              0.026806                        97.286289
149    0.269238              0.026272                        97.312561

[150 rows x 3 columns]


In [ ]:
# todo Umformung der ersten 12 Eigengesichter

### d) Von den Testbildern wird nun ebenfalls der Mittelwert der Trainingsdaten abgezogen (s. Schritt 1 im PCA-Algorithmus). Projizieren Sie jedes der Trainings- und Testbilder auf die ersten 7 Eigengesichter, d.h. Sie erhalten so für jedes Trainings- und Testbild 7 Merkmale. Die Gesichtserkennung geschieht nun dadurch, dass Sie den euklidischen Abstand des Testbildes in diesem 7-dimensionalen Merkmalsraum zu allen Trainingsbildern berechnen. Die Person des am nächsten liegenden Trainingsbildes (d.h. mit dem minimalen euklidischen Abstand) ist dann (vermutlich) auch die korrekte Person für das Testbild (Nächster-Nachbar-Klassifikator). Welche Bilder werden korrekt klassifiziert, welche Verwechslungen gibt es?